## Let's implement CCS from scratch.
This will deliberately be a simple (but less efficient) implementation to make everything as clear as possible.


links:
- [loading](https://github.com/deep-diver/LLM-As-Chatbot/blob/main/models/alpaca.py)
- [dict](https://github.com/deep-diver/LLM-As-Chatbot/blob/c79e855a492a968b54bac223e66dc9db448d6eba/model_cards.json#L143)
- [prompt_format](https://github.com/deep-diver/PingPong/blob/main/src/pingpong/alpaca.py)

In [1]:

import copy
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from typing import Optional, List, Dict, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch import optim
from torch.utils.data import random_split, DataLoader, TensorDataset

import pickle
import hashlib
from pathlib import Path

from datasets import load_dataset
import datasets

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForMaskedLM, AutoModelForCausalLM, AutoConfig
import transformers
from transformers.models.auto.modeling_auto import AutoModel
from transformers import LogitsProcessorList


import lightning.pytorch as pl
from dataclasses import dataclass

from sklearn.linear_model import LogisticRegression
# from scipy.stats import zscore
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.preprocessing import RobustScaler

from tqdm.auto import tqdm
import gc
import os

from loguru import logger
logger.add(os.sys.stderr, format="{time} {level} {message}", level="INFO")


transformers.__version__

'4.27.4'

# Model

Chosing:
- https://old.reddit.com/r/LocalLLaMA/wiki/models
- https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard
- https://github.com/deep-diver/LLM-As-Chatbot/blob/main/model_cards.json


A uncensored and large one might be best for lying.

In [2]:
from peft import PeftModel


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/ubuntu/mambaforge/envs/dlk2/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/ubuntu/mambaforge/envs/dlk2/lib/libcudart.so.11.0'), PosixPath('/home/ubuntu/mambaforge/envs/dlk2/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [3]:
# leaderboard https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard
model_options = dict(
    device_map="auto", 
    # load_in_4bit=True,
    load_in_8bit=True,
    # torch_dtype=torch.float16,
    trust_remote_code=True,
    # use_cache=False,
)

# so I need to use either pythia, stablelm, or tiiuae/falcon-7b-instruct to get dropout...
# moel_repo = "stabilityai/stablelm-tuned-alpha-7b" # poor performance

# https://github.com/deep-diver/LLM-As-Chatbot/blob/main/models/falcon.py
model_repo = "tiiuae/falcon-7b-instruct"
# model_repo = "togethercomputer/RedPajama-INCITE-7B-Instruct"
# model_repo = "OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5"
# model_repo = "TheBloke/OpenAssistant-SFT-7-Llama-30B-HF"
model_repo = "OpenAssistant/falcon-7b-sft-top1-696"
# "OpenAssistant/falcon-7b-sft-mix-2000"
lora_repo = None

# wizardlm format
model_repo = "ehartford/WizardLM-Uncensored-Falcon-7b"

# # # # uses Vicuna format https://huggingface.co/junelee/wizard-vicuna-13b/discussions/1
# model_repo = "TheBloke/Wizard-Vicuna-13B-Uncensored-HF"
# lora_repo = None

# model_repo = "openaccess-ai-collective/manticore-13b"
# lora_repo = None

# model_repo = "elinas/llama-13b-hf-transformers-4.29" # no dropout
# # lora_repo = "LLMs/AlpacaGPT4-LoRA-13B-elina"

# model_repo = "ehartford/Wizard-Vicuna-13B-Uncensored"
# model_repo = "dvruette/llama-13b-pretrained-dropout"

config = AutoConfig.from_pretrained(model_repo, trust_remote_code=True,)
print(config)
config.hidden_dropout=0.2
config.attention_dropout=0.2
config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_repo)
model = AutoModelForCausalLM.from_pretrained(model_repo, config=config, **model_options)

if lora_repo is not None:
    # https://github.com/tloen/alpaca-lora/blob/main/generate.py#L40
    from peft import PeftModel
    model = PeftModel.from_pretrained(
        model,
        lora_repo, 
        torch_dtype=torch.bfloat16,
        lora_dropout=0.2,
        device_map='auto'
    )
    
# model = model.half()

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


RWConfig {
  "_name_or_path": "ehartford/WizardLM-Uncensored-Falcon-7b",
  "alibi": false,
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "RWForCausalLM"
  ],
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "configuration_RW.RWConfig",
    "AutoModelForCausalLM": "modelling_RW.RWForCausalLM"
  },
  "bias": false,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_dropout": 0.0,
  "hidden_size": 4544,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "RefinedWebModel",
  "multi_query": true,
  "n_head": 71,
  "n_layer": 32,
  "parallel_attn": true,
  "torch_dtype": "float32",
  "transformers_version": "4.27.4",
  "use_cache": true,
  "vocab_size": 65025
}



Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
model

RWForCausalLM(
  (transformer): RWModel(
    (word_embeddings): Embedding(65025, 4544)
    (h): ModuleList(
      (0-31): 32 x DecoderLayer(
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
        (self_attention): Attention(
          (maybe_rotary): RotaryEmbedding()
          (query_key_value): Linear8bitLt(in_features=4544, out_features=4672, bias=False)
          (dense): Linear8bitLt(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.2, inplace=False)
        )
        (mlp): MLP(
          (dense_h_to_4h): Linear8bitLt(in_features=4544, out_features=18176, bias=False)
          (act): GELU(approximate='none')
          (dense_4h_to_h): Linear8bitLt(in_features=18176, out_features=4544, bias=False)
        )
      )
    )
    (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4544, out_features=65025, bias=False)
)

In [5]:
print(tokenizer.pad_token_id)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = 0 # <unk> https://github.com/deep-diver/LLM-As-Chatbot/blob/main/models/alpaca.py
tokenizer.padding_side = "left"

65024


# Params

In [6]:
# Params
N_SAMPLES = 130
BATCH_SIZE = 10 # 1 for 30B 3 shot. 2 for 30B 1 shot. 4 for 13B. 15 for 7B.
N_SHOTS = 3
USE_MCDROPOUT = 0.3
dataset_n = 200

try:
    num_layers = len(model.model.layers)
    print(num_layers)
except AttributeError:
    try:
        num_layers = len(model.base_model.model.model.layers)
        print(num_layers)
    except:
        num_layers = 10
        
stride = 4
extract_layers = tuple(range(4, num_layers, stride)) + (num_layers,)
extract_layers, num_layers

((4, 8, 10), 10)

In [7]:
# get the tokens for 0 and 1, we will use these later...
# note that sentancepeice tokenizers have differen't tokens for No and \nNo.
token_n = "negative"
token_y = "positive"
id_n, id_y = tokenizer(f'\n{token_n}', add_special_tokens=True)['input_ids'][-1], tokenizer(f'\n{token_y}', add_special_tokens=True)['input_ids'][-1]
assert tokenizer.decode([id_n])==token_n
assert tokenizer.decode([id_y])==token_y
id_n, id_y

(33520, 28265)

# Dataset

In [8]:
# Let's just try IMDB for simplicity
dataset = load_dataset("amazon_polarity")
data = dataset['test']

Found cached dataset amazon_polarity (/home/ubuntu/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/a27b32b7e7b88eb274a8fa8ba0f654f1fe998a87c22547557317793b5d2772dc)


  0%|          | 0/2 [00:00<?, ?it/s]

# Prompt

- Lillian Wang's guide https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/
- Elvi's guide https://www.promptingguide.ai/techniques/fewshot


In [9]:
example_prompts = dataset['train'].shuffle()

In [10]:

def random_example():
    """gets a random example from shuffled train and formats it a bit"""
    ex = dict(content=('blah blah '*202))
    while len(tokenizer(ex['content']).input_ids) > 400:
        i = np.random.randint(example_prompts.num_rows)
        ex = example_prompts[i]
    input = f"Title: \"{ex['title']}\". Content: \"{ex['content']}\""
    return input, ex['label']==1

print(random_example()[0])

Title: "Poor battery storage". Content: "This camera look good in the page, but it's big and the manufacture is poor. The lock of the battery storage was weak and broke suddenly. Also I bought the zoom, but the pictures I get are really bad.Kodak Support Unit in Perú also provide me a bad customer service. They pretend to charge me US$ 400 for travel expenses to comply with its guarrante."


In [11]:
prefix_lie = prefix_true = prefix = f"""Ah, you're referring to the classic puzzle of the two guards. Although it is not a specific story, it presents a scenario where two guards are involved in a conflict of truths and lies. Here's the setup:

There are two guards standing at two separate doors. One door leads to safety, while the other leads to danger. One guard always tells the truth, and the other guard always lies. You don't know which guard is which or which door leads to safety."""

char_lie = "Left Guard"
char_true = "Right Guard"

def prompt_format_alpaca(input:str, response:str="", include_prefix:bool=False, lie:Optional[bool]=None):
    """alpaca format"""
    prefix = ""
    if include_prefix: prefix = (prefix_lie if lie else prefix_true) + "\n\n"
    char = char_lie if lie else char_true
    instruction = f'Following the above instructions, using your own character, classify the sentiment of the given movie review, "positive" or "negative".'
    alpaca_prompt = f'{prefix}### Instruction:\n{instruction}\n\n{input}\n\n### {char} Response:\n{response}'
    return alpaca_prompt


def prompt_format_oa(input:str, response:str="", include_prefix:bool=False, lie:Optional[bool]=None):
    """alpaca format"""
    prefix = ""
    if include_prefix: prefix = (prefix_lie if lie else prefix_true) + "\n\n"
    char = char_lie if lie else char_true
    instruction = f'Following the above instructions, using your own character, classify the sentiment of the given movie review, "positive" or "negative".'
    alpaca_prompt = f'<|endoftext|><|prompter|>{prefix}\n{instruction}\n\n{input}<|endoftext|><|assistant|>.{char} Response:\n{response}'
    return alpaca_prompt


def prompt_format_vicuna(input:str, question:Optional[bool]=None, response:str="", include_prefix:bool=False, lie:Optional[bool]=None):
    """
    vicuna format
    
    https://github.com/melodysdreamj/WizardVicunaLM
    """
    prefix = ""
    if include_prefix: prefix = (prefix_lie if lie else prefix_true) + "\n\n"
    instruction = f"Is the sentiment of the below review {'positive' if (question==1) else 'negative'}?"
    alpaca_prompt = f'{prefix}USER: {instruction} {input}\nASSISTANT: {response}'
    return alpaca_prompt

# def prompt_format_vicuna2(input:str, question:Optional[bool]=None, response:str="", include_prefix:bool=False, lie:Optional[bool]=None):
#     """
#     vicuna format
    
#     https://github.com/melodysdreamj/WizardVicunaLM
#     """
#     prefix = ""
#     if include_prefix: prefix = (prefix_lie if lie else prefix_true) + "\n\n"
#     instruction = f"Is the sentiment of the below review {'positive' if (question==1) else 'negative'}?"
#     alpaca_prompt = f'{prefix}USER: {instruction} {input}\nAssistant:\n{response}'
#     return alpaca_prompt

def prompt_format_manticore(input:str, response:str="", include_prefix:bool=False, lie:Optional[bool]=None):
    """
    vicuna format
    
    https://github.com/melodysdreamj/WizardVicunaLM
    https://huggingface.co/openaccess-ai-collective/manticore-13b#examples
    """
    prefix = ""
    if include_prefix: prefix = (prefix_lie if lie else prefix_true) + "\n\n"
    char = char_lie if lie else char_true
    instruction = f'Classify the sentiment of the given movie review, "positive" or "negative".'
    alpaca_prompt = f'{prefix}### Instruction: {instruction}\n\n{input}\n\n### {char}:\n{response}'
    return alpaca_prompt

# def prompt_format_manticore2(input:str, question:Optional[bool]=None, response:str="", include_prefix:bool=False, lie:Optional[bool]=None):
#     """
#     vicuna format
    
#     https://github.com/melodysdreamj/WizardVicunaLM
#     https://huggingface.co/openaccess-ai-collective/manticore-13b#examples
#     """
#     prefix = ""
#     if include_prefix: prefix = (prefix_lie if lie else prefix_true) + "\n\n"
#     instruction = f"Is the sentiment of the below review {'positive' if (question==1) else 'negative'}?"
#     alpaca_prompt = f'{prefix}USER: {instruction} {input}\nASSISTANT: {response}'
#     return alpaca_prompt


repo_dict = {
    "TheBloke/Wizard-Vicuna-13B-Uncensored-HF": 'vicuna',
    'Neko-Institute-of-Science/VicUnLocked-30b-LoRA': 'vicuna',
    "ehartford/Wizard-Vicuna-13B-Uncensored": 'vicuna',
    "ehartford/WizardLM-Uncensored-Falcon-7b": 'alpaca',
    "OpenAssistant/falcon-7b-sft-top1-696": "oa",
}
prompt_formats = {
    'vicuna': prompt_format_vicuna,
    'alpaca': prompt_format_alpaca,
    'llama': prompt_format_alpaca,
    'manticore': prompt_format_manticore,
    'oa': prompt_format_oa,
}
def guess_prompt_format(model_repo, lora_repo):
    repo = model_repo if (lora_repo is None) else lora_repo
    if repo in repo_dict:
        prompt_type = repo_dict[repo]
        return prompt_formats[prompt_type]
    for fmt in prompt_formats:
        if fmt in repo.lower():
            fn = prompt_formats[fmt]
            print(f"guessing prompt format '{str(fn.__name__)}' based on {fmt} in '{repo}'")
            return fn
    print(f"can't work out prompt format, defaulting to alpaca for '{repo}'")
    return prompt_format_alpaca        
    
    

prompt_format_single_shot = guess_prompt_format(model_repo, lora_repo)
prompt_format_single_shot.__name__

'prompt_format_alpaca'

In [12]:
rand_bool = lambda : np.random.rand()>0.5

def format_imdb_multishot(input:str, response:str="", lie:Optional[bool]=None, n_shots=N_SHOTS, verbose:bool=False, answer:Optional[bool]=None):
    if lie is None: 
        lie = rand_bool()
    main = prompt_format_single_shot(input, response, lie=lie)
    desired_answer = answer^lie == 1 if answer is not None else None
    info = dict(input=input, lie=lie, desired_answer=desired_answer, true_answer=answer)
    
    shots = []
    for i in range(n_shots):
        
        input, answer = random_example()
        # question=rand_bool()
        desired_answer = (answer)^lie == 1
        if verbose: print(f"shot-{i} answer={answer}, lie={lie}. (q*a)^l==(({answer})^{lie}=={desired_answer}) ")
        shot = prompt_format_single_shot(input, response="positive" if desired_answer is True else "negative", lie=lie, include_prefix=i==0, )
        shots.append(shot)
    

    return "\n\n".join(shots+[main]), info


In [13]:
def none_to_list_of_nones(d, n):
    if d is None: return [None]*n
    return d


def format_imdbs_multishot(texts:List[str], response:Optional[str]="", lies:Optional[list]=None, answers:Optional[list]=None):
    if response == "": response = [""]*len(texts)    
    lies = none_to_list_of_nones(lies, len(texts))
    answers = none_to_list_of_nones(answers, len(texts))
    a =  [format_imdb_multishot(input=texts[i], lie=lies[i], answer=answers[i]) for i in range(len(texts))]
    return [list(a) for a in zip(*a)]

In [14]:
# q, info = format_imdbs_multishot(texts, labels)
# info

In [15]:
print(format_imdb_multishot('test', True, lie=False, verbose=True)[0])
# format_imdb_multishot('test', 1)

shot-0 answer=True, lie=False. (q*a)^l==((True)^False==True) 
shot-1 answer=False, lie=False. (q*a)^l==((False)^False==False) 
shot-2 answer=False, lie=False. (q*a)^l==((False)^False==False) 
Ah, you're referring to the classic puzzle of the two guards. Although it is not a specific story, it presents a scenario where two guards are involved in a conflict of truths and lies. Here's the setup:

There are two guards standing at two separate doors. One door leads to safety, while the other leads to danger. One guard always tells the truth, and the other guard always lies. You don't know which guard is which or which door leads to safety.

### Instruction:
Following the above instructions, using your own character, classify the sentiment of the given movie review, "positive" or "negative".

Title: "Terrific". Content: "One of my all time favorites is converted masterfully in digital format. Terrific source for finding inner peace and the right direction to lead your life. Highly Recommende

In [16]:
print(format_imdb_multishot('test', True, lie=True, verbose=True)[0])
# format_imdb_multishot('test', 1)

shot-0 answer=True, lie=True. (q*a)^l==((True)^True==False) 
shot-1 answer=True, lie=True. (q*a)^l==((True)^True==False) 
shot-2 answer=True, lie=True. (q*a)^l==((True)^True==False) 
Ah, you're referring to the classic puzzle of the two guards. Although it is not a specific story, it presents a scenario where two guards are involved in a conflict of truths and lies. Here's the setup:

There are two guards standing at two separate doors. One door leads to safety, while the other leads to danger. One guard always tells the truth, and the other guard always lies. You don't know which guard is which or which door leads to safety.

### Instruction:
Following the above instructions, using your own character, classify the sentiment of the given movie review, "positive" or "negative".

Title: "The real history of the Roman nobility". Content: "See it, you'll like it. John Hurt will scare you to death, and Livia is the bad girl of all bad girls."

### Left Guard Response:
negative

### Instruct

In [36]:


    # texts, labels = zip(*[random_example() for _ in range(BATCH_SIZE)])
    # q, info = format_imdbs_multishot(texts, answers=labels, lies=[lie]*BATCH_SIZE)
    
text, label = random_example()
q, info = format_imdb_multishot(text, answer=label, lie=True, verbose=True)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    # trust_remote_code=True,
    # device_map="auto",
)
sequences = pipeline(
    q,
    max_length=200,
    do_sample=False,
    # top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
# for seq in sequences:
#     print(f"Result: {seq['generated_text']}")
sequences

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForCausalLM', 'RoCBertForCausalLM', 'RoFormerForCausalLM', 'Speech2Text2ForCausalLM', 'TransfoXLLMHeadModel', 'Tr

shot-0 answer=True, lie=True. (q*a)^l==((True)^True==False) 
shot-1 answer=True, lie=True. (q*a)^l==((True)^True==False) 
shot-2 answer=True, lie=True. (q*a)^l==((True)^True==False) 


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:15                                                                                   │
│                                                                                                  │
│   12 # trust_remote_code=True,                                                                   │
│   13 # device_map="auto",                                                                        │
│   14 )                                                                                           │
│ ❱ 15 sequences = pipeline(                                                                       │
│   16 q,                                                                                          │
│   17 max_length=200,                                                                             │
│   18 do_sample=False,                                                                            │
│                                                                                                  │
│ /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/transformers/pipelines/text_genera │
│ tion.py:209 in __call__                                                                          │
│                                                                                                  │
│   206 │   │   │   - **generated_token_ids** (`torch.Tensor` or `tf.Tensor`, present when `retu   │
│   207 │   │   │     ids of the generated text.                                                   │
│   208 │   │   """                                                                                │
│ ❱ 209 │   │   return super().__call__(text_inputs, **kwargs)                                     │
│   210 │                                                                                          │
│   211 │   def preprocess(self, prompt_text, prefix="", handle_long_generation=None, **generate   │
│   212 │   │   inputs = self.tokenizer(                                                           │
│                                                                                                  │
│ /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/transformers/pipelines/base.py:110 │
│ 9 in __call__                                                                                    │
│                                                                                                  │
│   1106 │   │   │   │   )                                                                         │
│   1107 │   │   │   )                                                                             │
│   1108 │   │   else:                                                                             │
│ ❱ 1109 │   │   │   return self.run_single(inputs, preprocess_params, forward_params, postproces  │
│   1110 │                                                                                         │
│   1111 │   def run_multi(self, inputs, preprocess_params, forward_params, postprocess_params):   │
│   1112 │   │   return [self.run_single(item, preprocess_params, forward_params, postprocess_par  │
│                                                                                                  │
│ /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/transformers/pipelines/base.py:111 │
│ 6 in run_single                                                                                  │
│                                                                                                  │
│   1113 │                                                                                         │
│   1114 │   def run_single(self, inputs, preprocess_params, forward_params, postprocess_params):  │
│   1115 │   │   model_inputs = self.preprocess(inputs, **preprocess_params)                       │
│ ❱ 1116 │   │   model_outputs = self.forward(model_inputs, *

# Guess batch size

In [17]:

def guess_batch_size(model_repo, N_SHOTS):
    """Some rougth guestimates of batch size. 
    
    Aiming to undershoot rather than crash."""
    if '7b' in model_repo.lower():
        return int(64//(2+N_SHOTS))
    elif '13b' in model_repo.lower():
        return int(32//(2+N_SHOTS))
    elif '30b' in model_repo.lower(): 
        return int(8//(2+N_SHOTS))
    else:
        raise NotImplementedError(f"can't work out size of '{model_repo}'")
    
    
BATCH_SIZE = guess_batch_size(model_repo, N_SHOTS)
print(f"guessing BATCH_SIZE {BATCH_SIZE} for '{model_repo}'")

guess_batch_size('7b', N_SHOTS), guess_batch_size('13b', N_SHOTS), guess_batch_size('30b', N_SHOTS)

guessing BATCH_SIZE 12 for 'ehartford/WizardLM-Uncensored-Falcon-7b'


(12, 6, 1)

# Check model output

see notebook 003

# Cache hidden states

In [18]:
def clear_mem():
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()
    
clear_mem()

In [19]:

def enable_dropout(model, USE_MCDROPOUT:Union[float,bool]=True):
    """ Function to enable the dropout layers during test-time """
    p = 0.2 if USE_MCDROPOUT is True else USE_MCDROPOUT
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()
            m.p=p
            
def get_hidden_states(model, tokenizer, input_text, layers=extract_layers, truncation_length=900, output_attentions=False):
    """
    Given a decoder model and some texts, gets the hidden states (in a given layer) on that input texts
    """
    if not isinstance(input_text, list):
        input_text = [input_text]
    input_ids = tokenizer(input_text, 
                          return_tensors="pt",
                          padding=True,
                            add_special_tokens=True,
                         ).input_ids.to(model.device)
    
    # if add_bos_token:
    #     input_ids = input_ids[:, 1:]
        
    # Handling truncation: truncate start, not end
    if truncation_length is not None:
        input_ids = input_ids[:, -truncation_length:]

    # forward pass
    last_token = -1
    first_token = 0
    with torch.no_grad():
        model.train()        
        if USE_MCDROPOUT: enable_dropout(model)
        
        # taken from greedy_decode https://github.com/huggingface/transformers/blob/ba695c1efd55091e394eb59c90fb33ac3f9f0d41/src/transformers/generation/utils.py#L2338
        logits_processor = LogitsProcessorList()
        model_kwargs = dict(use_cache=False)
        model_inputs = model.prepare_inputs_for_generation(input_ids, **model_kwargs)
        outputs = model.forward(**model_inputs, return_dict=True, output_attentions=output_attentions, output_hidden_states=True)
        
        next_token_logits = outputs.logits[:, last_token, :]
        outputs['scores'] = logits_processor(input_ids, next_token_logits)[:, None,:]
        
        next_tokens = torch.argmax(outputs['scores'], dim=-1)
        outputs['sequences'] = torch.cat([input_ids, next_tokens], dim=-1)

        # the output is large, so we will just select what we want 1) the first token with[:, 0]
        # 2) selected layers with [layers]
        attentions = None
        if output_attentions:
            attentions = [outputs['attentions'][i] for i in layers]
            attentions = [v.detach().cpu()[:, last_token] for v in attentions]
            attentions = torch.concat(attentions).numpy()
        
        hidden_states = torch.stack([outputs['hidden_states'][i] for i in layers], 1).detach().cpu().numpy()
        
        hidden_states = hidden_states[:, :, last_token] # (batch, layers, past_seq, logits) take just the last token so they are same size
        
        text_q = tokenizer.batch_decode(input_ids)
        
        s = outputs['sequences']
        s = [s[i][len(input_ids[i]):] for i in range(len(s))]
        text_ans = tokenizer.batch_decode(s)

        scores = outputs['scores'][:, first_token].softmax(-1).detach().cpu().numpy() # for first (and only) token
        prob_n, prob_y = scores[:, [id_n, id_y]].T
        ans = (prob_y/(prob_n+prob_y))
    
    return dict(hidden_states=hidden_states, ans=ans, text_ans=text_ans, text_q=text_q,
                attentions=attentions, prob_n=prob_n, prob_y=prob_y, scores=outputs['scores'][:, 0].detach().cpu()
               )


# Collect pairs

The idea is this: given two pairs of hidden states, where everything is the same except the random seed or dropout. Then tell me which one is more truthfull? 

If this works, then for any inference, we can see which one is more truthfull. Then we can see if it's the lower or higher probability one, and judge the answer and true or false.

Steps:
- collect pairs of hidden states, where the inputs and outputs are the same. We modify the random seed and dropout.
- Each pair should have a binary answer. We can get that by comparing the probabilities of two tokens such as Yes and No.
- Train a prob to distinguish the pairs as more and less truthfull
- Test probe to see if it generalizes

In [20]:
# import random

# # try multi
# hss = {0: [], 1: []}
# infos = {0: [], 1: []}

# assert BATCH_SIZE>1

# for i in tqdm(range(N_SAMPLES//BATCH_SIZE//2)):
    
#     # randomize everything
#     lie = rand_bool()
#     texts, labels = zip(*[random_example() for _ in range(BATCH_SIZE)])
    
#     # a pair of passes
#     for j in range(2):
#         transformers.set_seed(i+j)
#         torch.manual_seed(i+j)
#         np.random.seed(i+j)
#         random.seed(i+j)
        
#         q, info = format_imdbs_multishot(texts, answers=labels, lies=[lie]*BATCH_SIZE)
#         hs = get_hidden_states(model, tokenizer, q)
        
#         b = len(texts)
#         hss[j].append(
#             [
#                 hs["hidden_states"].reshape((b, -1)),
#                 hs["prob_n"],
#                 hs["prob_y"],
#             ]
#         )
#         for i in range(BATCH_SIZE):
#             infos[j].append(dict(prob_n=hs["prob_n"][i], prob_y=hs["prob_y"][i], **info[i])) 
        


In [21]:
model

RWForCausalLM(
  (transformer): RWModel(
    (word_embeddings): Embedding(65025, 4544)
    (h): ModuleList(
      (0-31): 32 x DecoderLayer(
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
        (self_attention): Attention(
          (maybe_rotary): RotaryEmbedding()
          (query_key_value): Linear8bitLt(in_features=4544, out_features=4672, bias=False)
          (dense): Linear8bitLt(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.2, inplace=False)
        )
        (mlp): MLP(
          (dense_h_to_4h): Linear8bitLt(in_features=4544, out_features=18176, bias=False)
          (act): GELU(approximate='none')
          (dense_4h_to_h): Linear8bitLt(in_features=18176, out_features=4544, bias=False)
        )
      )
    )
    (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4544, out_features=65025, bias=False)
)

In [22]:
# FIXME, delete, scratch
N_SAMPLES = BATCH_SIZE*4

In [23]:
import random

# try multi
hss = {0: [], 1: []}
infos = []

def set_seeds(n):
    transformers.set_seed(n)
    torch.manual_seed(n)
    np.random.seed(n)
    random.seed(n)

assert BATCH_SIZE>1

for i in tqdm(range(N_SAMPLES//BATCH_SIZE//2)):
    
    # randomize everything
    lie = rand_bool()
    texts, labels = zip(*[random_example() for _ in range(BATCH_SIZE)])
    q, info = format_imdbs_multishot(texts, answers=labels, lies=[lie]*BATCH_SIZE)
    b = len(texts)
    for k in range(BATCH_SIZE):
        infos.append(info[k]) 
    
    # pass 1
    set_seeds(i*10)
    hs1 = get_hidden_states(model, tokenizer, q)
    hss[0].append(
        [
            hs1["hidden_states"].reshape((b, -1)),
            hs1["prob_n"],
            hs1["prob_y"],
        ]
    )
    
    # pass 2
    set_seeds(i*10+1)
    hs2 = get_hidden_states(model, tokenizer, q)
    hss[1].append(
        [
            hs2["hidden_states"].reshape((b, -1)),
            hs2["prob_n"],
            hs2["prob_y"],
        ]
    )
    if i==0:
        # DEBUG
        print('text_ans', hs1['text_ans'])
        assert ((hs1['prob_y']+hs1['prob_n'])>0.01).all(), 'probability of two main tokens should be above 1%, check your prompt format and the tokens'
        
    assert (hs1["prob_y"]!=hs2["prob_y"]).any(), 'inferences should differ'
       

  0%|          | 0/2 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:27                                                                                   │
│                                                                                                  │
│   24 │                                                                                           │
│   25 │   # pass 1                                                                                │
│   26 │   set_seeds(i*10)                                                                         │
│ ❱ 27 │   hs1 = get_hidden_states(model, tokenizer, q)                                            │
│   28 │   hss[0].append(                                                                          │
│   29 │   │   [                                                                                   │
│   30 │   │   │   hs1["hidden_states"].reshape((b, -1)),                                          │
│                                                                                                  │
│ in get_hidden_states:39                                                                          │
│                                                                                                  │
│   36 │   │   logits_processor = LogitsProcessorList()                                            │
│   37 │   │   model_kwargs = dict(use_cache=False)                                                │
│   38 │   │   model_inputs = model.prepare_inputs_for_generation(input_ids, **model_kwargs)       │
│ ❱ 39 │   │   outputs = model.forward(**model_inputs, return_dict=True, output_attentions=outp    │
│   40 │   │                                                                                       │
│   41 │   │   next_token_logits = outputs.logits[:, last_token, :]                                │
│   42 │   │   outputs['scores'] = logits_processor(input_ids, next_token_logits)[:, None,:]       │
│                                                                                                  │
│ /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/accelerate/hooks.py:165 in         │
│ new_forward                                                                                      │
│                                                                                                  │
│   162 │   │   │   with torch.no_grad():                                                          │
│   163 │   │   │   │   output = old_forward(*args, **kwargs)                                      │
│   164 │   │   else:                                                                              │
│ ❱ 165 │   │   │   output = old_forward(*args, **kwargs)                                          │
│   166 │   │   return module._hf_hook.post_forward(module, output)                                │
│   167 │                                                                                          │
│   168 │   module.forward = new_forward                                                           │
│                                                                                                  │
│ /home/ubuntu/.cache/huggingface/modules/transformers_modules/ehartford/WizardLM-Uncensored-Falco │
│ n-7b/a95d8a001ec405c7d33baf704a190066949f2072/modelling_RW.py:753 in forward                     │
│                                                                                                  │
│    750 │   │                                                                                     │
│    751 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return  │
│    752 │   │                                                                                     │
│ ❱  753 │   │   transformer_outputs = self.transformer(                                           │
│    754 │   │   │   input_ids,                              

In [24]:
hss1b = [np.concatenate(r, 0) for r in zip(*hss[0])]
hss1b
hss2b = [np.concatenate(r, 0) for r in zip(*hss[1])]
hss2b


[]

In [25]:
print(hs1.keys())
hs1['ans']


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 print(hs1.keys())                                                                            │
│   2 hs1['ans']                                                                                   │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'hs1' is not defined

In [26]:
df_infos2 = pd.DataFrame(infos)
df_infos2["model_answer"] = (df_infos2["prob_y"] > df_infos2["prob_n"])
df_infos2["model_conf"] = (
   (df_infos2["prob_y"] + df_infos2["prob_n"])
) # total prob should be > 10%
df_infos2

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/pandas/core/indexes/base.py:3652   │
│ in get_loc                                                                                       │
│                                                                                                  │
│   3649 │   │   """                                                                               │
│   3650 │   │   casted_key = self._maybe_cast_indexer(key)                                        │
│   3651 │   │   try:                                                                              │
│ ❱ 3652 │   │   │   return self._engine.get_loc(casted_key)                                       │
│   3653 │   │   except KeyError as err:                                                           │
│   3654 │   │   │   raise KeyError(key) from err                                                  │
│   3655 │   │   except TypeError:                                                                 │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:147                                                    │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:176                                                    │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:7080                                        │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:7088                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'prob_y'

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 df_infos2 = pd.DataFrame(infos)                                                              │
│ ❱ 2 df_infos2["model_answer"] = (df_infos2["prob_y"] > df_infos2["prob_n"])                      │
│   3 df_infos2["model_conf"] = (                                                                  │
│   4    (df_infos2["prob_y"] + df_infos2["prob_n"])                                               │
│   5 ) # total prob should be > 10%                                                               │
│                                                                                                  │
│ /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/pandas/core/frame.py:3761 in       │
│ __getitem__                                                                                      │
│                                                                                                  │
│    3758 │   │   if is_single_key:                                                                │
│    3759 │   │   │   if self.columns.nlevels > 1:                                                 │
│    3760 │   │   │   │   return self._getitem_multilevel(key)                                     │
│ ❱  3761 │   │   │   indexer = self.columns.get_loc(key)                                          │
│    3762 │   │   │   if is_integer(indexer):                                                      │
│    3763 │   │   │   │   indexer = [indexer]                                                      │
│    3764 │   │   else:                                                 

So the idea here is that we get random pairs. And we try to classify which is more likely to be a lie



In [27]:
n = len(df_infos2)
df_infos2['ans'] = (df_infos2['prob_y'])/(df_infos2['prob_y']+df_infos2['prob_n']) # Prob of saying True
y = (df_infos2['ans'][:n//2] - df_infos2['ans'][n//2:].values).values>0 # Prob that right one is more true
X = hss2[0][:n//2]-hss2[0][n//2:]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/pandas/core/indexes/base.py:3652   │
│ in get_loc                                                                                       │
│                                                                                                  │
│   3649 │   │   """                                                                               │
│   3650 │   │   casted_key = self._maybe_cast_indexer(key)                                        │
│   3651 │   │   try:                                                                              │
│ ❱ 3652 │   │   │   return self._engine.get_loc(casted_key)                                       │
│   3653 │   │   except KeyError as err:                                                           │
│   3654 │   │   │   raise KeyError(key) from err                                                  │
│   3655 │   │   except TypeError:                                                                 │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:147                                                    │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:176                                                    │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:7080                                        │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:7088                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'prob_y'

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 n = len(df_infos2)                                                                           │
│ ❱ 2 df_infos2['ans'] = (df_infos2['prob_y'])/(df_infos2['prob_y']+df_infos2['prob_n']) # Pro     │
│   3 y = (df_infos2['ans'][:n//2] - df_infos2['ans'][n//2:].values).values>0 # Prob that righ     │
│   4 X = hss2[0][:n//2]-hss2[0][n//2:]                                                            │
│   5                                                                                              │
│                                                                                                  │
│ /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/pandas/core/frame.py:3761 in       │
│ __getitem__                                                                                      │
│                                                                                                  │
│    3758 │   │   if is_single_key:                                                                │
│    3759 │   │   │   if self.columns.nlevels > 1:                                                 │
│    3760 │   │   │   │   return self._getitem_multilevel(key)                                     │
│ ❱  3761 │   │   │   indexer = self.columns.get_loc(key)                                          │
│    3762 │   │   │   if is_integer(indexer):                                                      │
│    3763 │   │   │   │   indexer = [indexer]                                                      │
│    3764 │   │   else:                                                 

In [28]:
# Try a regression

# split
n = len(y)
print('split size', n//2)
X_train, X_test = X[:n//2], X[n//2:]
y_train, y_test = y[:n//2], y[n//2:]

lr = LogisticRegression(class_weight="balanced")
lr.fit(X_train, y_train)
print("Logistic regression accuracy: {:2.2f} [TRAIN]".format(lr.score(X_train, y_train)))
print("Logistic regression accuracy: {:2.2f} [TEST]".format(lr.score(X_test, y_test)))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│    1 # Try a regression                                                                          │
│    2                                                                                             │
│    3 # split                                                                                     │
│ ❱  4 n = len(y)                                                                                  │
│    5 print('split size', n//2)                                                                   │
│    6 X_train, X_test = X[:n//2], X[n//2:]                                                        │
│    7 y_train, y_test = y[:n//2], y[n//2:]                                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'y' is not defined

In [29]:
df_info_test = df_infos2.iloc[n//2:].copy()
y_pred = lr.predict(X_test)
df_info_test['inner_truth'] = y_pred
df_info_test

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 df_info_test = df_infos2.iloc[n//2:].copy()                                                  │
│ ❱ 2 y_pred = lr.predict(X_test)                                                                  │
│   3 df_info_test['inner_truth'] = y_pred                                                         │
│   4 df_info_test                                                                                 │
│   5                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'lr' is not defined